In [1]:
%matplotlib 
#%matplotlib inline
#%matplotlib qt
import os
import csv
import numpy as np
import fnmatch
from pathlib import Path
import pandas as pd
import xlrd
import matplotlib.pyplot as plt
from scipy import stats 
import statsmodels
from statsmodels.graphics.gofplots import qqplot
import pickle


Using matplotlib backend: Qt5Agg


In [84]:
# first find the number of rows in the existing file

#subjectName = 'Per'
#subjectName = 'Torben'
subjectName = 'John'


if subjectName == 'Torben':
    hot_indices = ['0', '1', '2', '1', '2']
else:
    hot_indices = ['1', '2', '1', '2']

filePath_PythonData = r"C:\DTU\Data\PainExperiments\201907_PainTrial2\ExportedData\raw" + "\\" + subjectName + "_wEvents_raw.xlsx"


df_data = pd.read_excel(filePath_PythonData)

In [85]:
# extract pupil data

pupilRawLeft = df_data['Pupil diameter left [mm]'].tolist()
pupilRawRight = df_data['Pupil diameter right [mm]'].tolist()

pupilRawLeft_woNan = [pupilSize for ind, pupilSize in enumerate(pupilRawLeft) if not np.isnan(pupilSize) and not np.isnan(pupilRawRight[ind])]
pupilRawRight_woNan = [pupilSize for ind, pupilSize in enumerate(pupilRawRight) if not np.isnan(pupilSize) and not np.isnan(pupilRawLeft[ind])]


# extract event information
events = df_data['Event'].tolist()

# extract the logged events
indices_loggedEvents_all = [i for i, x in enumerate(events) if x == "Logged live Event"]


# extract time 
timeMicroS = df_data['Recording timestamp [μs]'].tolist()
timeMilliS = [time/1000 for time in timeMicroS]

timeMilliS_woNan = [timeMilliS[ind] for ind, pupilSize in enumerate(pupilRawLeft) if not np.isnan(pupilSize) and not np.isnan(pupilRawRight[ind])]




In [86]:
def hampel(vals_orig, k, sd):
    '''
    vals: pandas series of values from which to remove outliers
    k: size of window (including the sample; 7 is equal to 3 on either side of value)
    '''
    # Obtained from: https://stackoverflow.com/questions/46819260/filtering-outliers-how-to-make-median-based-
    # hampel-function-faster
    
    #Make copy so original not edited
    vals0 = pd.DataFrame(vals_orig)      
    vals1 = vals0.replace([np.inf, -np.inf], np.nan)
    
    vals2 = vals1.astype(float).interpolate('linear', limit_direction = 'both') # linear interpolation instead of 
    # simply copying the previous value --\ linear interpolation than cubic to not add any patterns in the data, limit direction
    # set to both, to interpolate the nan values occuring from the start of the series
    
    L= 1.4826
    # compte the rolling mean
    rolling_median = vals2.rolling(window=k, min_periods=1, center=True).median()
    
    difference = np.abs(rolling_median-vals2)
    median_abs_deviation = difference.rolling(k).median()
    threshold = sd * L * median_abs_deviation
    
    outlier_idx = difference>threshold
    
    #print(vals2[outlier_idx])
    
    vals2[outlier_idx] = rolling_median[outlier_idx]
    
    for i in vals2:
        if np.isnan(i):
            print(i)
    
    return(vals2)

## Pupil sizes from pin in the dark

In [87]:
# indexes of pin events
index_pin_start =  [ind for ind, row in enumerate(events) if row == 'pin_start']
index_pin_end = [ind for ind, row in enumerate(events) if row == 'pin_end']

# pupil sizes for pain from pin 
pupilRawLeft_pin_dark = pupilRawLeft[index_pin_start[0]:index_pin_end[0]]
pupilRawRight_pin_dark = pupilRawRight[index_pin_start[0]:index_pin_end[0]]
time_pin_dark = timeMilliS[index_pin_start[0]:index_pin_end[0]]

# pupil sizes from pin pain without nan
pupilRawLeft_pin_woNan_dark = [pupilSize for ind, pupilSize in enumerate(pupilRawLeft_pin_dark) if not np.isnan(pupilSize) and not np.isnan(pupilRawRight_pin_dark[ind])]
pupilRawRight_pin_woNan_dark = [pupilSize for ind, pupilSize in enumerate(pupilRawRight_pin_dark) if not np.isnan(pupilSize) and not np.isnan(pupilRawLeft_pin_dark[ind])]
time_pin_woNan_dark = [time_pin_dark[ind] for ind, pupilSize in enumerate(pupilRawLeft_pin_dark) if not np.isnan(pupilSize) and not np.isnan(pupilRawRight_pin_dark[ind])]

time_pin_woNan_dark_plot = [time/1000 for time in time_pin_woNan_dark]

In [88]:
# find the logged events in the pin events

for index in indices_loggedEvents_all:
    if index > index_pin_start[0]:
        logged_events_start = index
        break
        
for index in sorted(indices_loggedEvents_all, reverse=True):
    if index < index_pin_end[0]:
        logged_events_end = index
        break    
        
indices_loggedEvents_pin_dark = indices_loggedEvents_all[indices_loggedEvents_all.index(logged_events_start):indices_loggedEvents_all.index(logged_events_end)+1]

In [89]:
# filter the pupil size for pins in the dark using hampel filter 

winSize = 35
sd = 3

pupilLeft_pin_dark_filtered = hampel(pupilRawLeft_pin_woNan_dark, winSize, sd)
pupilRight_pin_dark_filtered = hampel(pupilRawRight_pin_woNan_dark, winSize, sd)

In [90]:
# check the filtered signal for the pins in the dark

fig = plt.figure()

ax = fig.add_subplot(111)

ax.plot(time_pin_woNan_dark_plot, pupilRawLeft_pin_woNan_dark, label = 'unfiltered')
ax.plot(time_pin_woNan_dark_plot, pupilLeft_pin_dark_filtered, label = 'filtered')

plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Absolute pupil size [in mm]')

ax.set_title('Check hampel filter')

pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName + '\\checkFiltering_leftPupil', 'wb'))



In [91]:
# plot the filtered signal for pins in the dark and mark the logged live events
fig = plt.figure()

ax = fig.add_subplot(111)

ax.plot(time_pin_woNan_dark_plot, pupilLeft_pin_dark_filtered, label = 'left')
ax.plot(time_pin_woNan_dark_plot, pupilRight_pin_dark_filtered, label = 'right')

plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Pupil size [in mm]')

ax.set_title('Subject: %s -- Pupil size from pins in the dark' %subjectName)

for ind in indices_loggedEvents_pin_dark:
    ax.plot([timeMilliS[ind]/1000, timeMilliS[ind]/1000], [np.min(pupilLeft_pin_dark_filtered), np.max(pupilLeft_pin_dark_filtered)], '-.', color = 'gray')

pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName +  '\\pupils_pin_dark_wLogEventsMarked', 'wb'))


## Pupil sizes from hot cup in the dark

In [92]:
# indexes for the pain using hot cup in the dark

# indexes of hot events
index_hot_start =  [ind for ind, row in enumerate(events) if row == 'hot_start']
index_hot_end = [ind for ind, row in enumerate(events) if row == 'hot_end']


# hand1
# indexes of hot events for hand1
index_hot_start_H1 = [index for ind, index in enumerate(index_hot_start) if hot_indices[ind] == '1'][0]
index_hot_end_H1 = [index for ind, index in enumerate(index_hot_end) if hot_indices[ind] == '1'][0]

# pupil sizes for pain from hot for hand1
pupilRawLeft_hot_dark_H1 = pupilRawLeft[index_hot_start_H1:index_hot_end_H1]
pupilRawRight_hot_dark_H1 = pupilRawRight[index_hot_start_H1:index_hot_end_H1]
time_hot_dark_H1 = timeMilliS[index_hot_start_H1:index_hot_end_H1]

# pupil sizes from hot pain without nan for hand1
pupilRawLeft_hot_woNan_dark_H1 = [pupilSize for ind, pupilSize in enumerate(pupilRawLeft_hot_dark_H1) if not np.isnan(pupilSize) and not np.isnan(pupilRawRight_hot_dark_H1[ind])]
pupilRawRight_hot_woNan_dark_H1 = [pupilSize for ind, pupilSize in enumerate(pupilRawRight_hot_dark_H1) if not np.isnan(pupilSize) and not np.isnan(pupilRawLeft_hot_dark_H1[ind])]
time_hot_woNan_dark_H1 = [time_hot_dark_H1[ind] for ind, pupilSize in enumerate(pupilRawLeft_hot_dark_H1) if not np.isnan(pupilSize) and not np.isnan(pupilRawRight_hot_dark_H1[ind])]

time_hot_woNan_dark_plot_H1 = [time/1000 for time in time_hot_woNan_dark_H1]




# hand2
# indexes of hot events for hand2
index_hot_start_H2 = [index for ind, index in enumerate(index_hot_start) if hot_indices[ind] == '2'][0]
index_hot_end_H2 = [index for ind, index in enumerate(index_hot_end) if hot_indices[ind] == '2'][0]

# pupil sizes for pain from hot for hand2
pupilRawLeft_hot_dark_H2 = pupilRawLeft[index_hot_start_H2:index_hot_end_H2]
pupilRawRight_hot_dark_H2 = pupilRawRight[index_hot_start_H2:index_hot_end_H2]
time_hot_dark_H2 = timeMilliS[index_hot_start_H2:index_hot_end_H2]

# pupil sizes from hot pain without nan for hand2
pupilRawLeft_hot_woNan_dark_H2 = [pupilSize for ind, pupilSize in enumerate(pupilRawLeft_hot_dark_H2) if not np.isnan(pupilSize) and not np.isnan(pupilRawRight_hot_dark_H2[ind])]
pupilRawRight_hot_woNan_dark_H2 = [pupilSize for ind, pupilSize in enumerate(pupilRawRight_hot_dark_H2) if not np.isnan(pupilSize) and not np.isnan(pupilRawLeft_hot_dark_H2[ind])]
time_hot_woNan_dark_H2 = [time_hot_dark_H2[ind] for ind, pupilSize in enumerate(pupilRawLeft_hot_dark_H2) if not np.isnan(pupilSize) and not np.isnan(pupilRawRight_hot_dark_H2[ind])]

time_hot_woNan_dark_plot_H2 = [time/1000 for time in time_hot_woNan_dark_H2]




In [93]:
# find the logged events in the hot events in the dark

# hand1
for index in indices_loggedEvents_all:
    if index > index_hot_start_H1:
        logged_events_start_H1 = index
        break
        
for index in sorted(indices_loggedEvents_all, reverse=True):
    if index < index_hot_end_H1:
        logged_events_end_H1 = index
        break

indices_loggedEvents_hot_dark_H1 = indices_loggedEvents_all[indices_loggedEvents_all.index(logged_events_start_H1):indices_loggedEvents_all.index(logged_events_end_H1)+1]


# hand2
for index in indices_loggedEvents_all:
    if index > index_hot_start_H2:
        logged_events_start_H2 = index
        break
        
for index in sorted(indices_loggedEvents_all, reverse=True):
    if index < index_hot_end_H2:
        logged_events_end_H2 = index
        break
        
indices_loggedEvents_hot_dark_H2 = indices_loggedEvents_all[indices_loggedEvents_all.index(logged_events_start_H2):indices_loggedEvents_all.index(logged_events_end_H2)+1]


In [106]:
print(indices_loggedEvents_hot_dark_H1)

[12504, 12554]


In [94]:
# filter the pupil size for hot cup in the dark using hampel filter 

winSize = 35
sd = 3

# hand1
pupilLeft_hot_dark_filtered_H1 = hampel(pupilRawLeft_hot_woNan_dark_H1, winSize, sd)
pupilRight_hot_dark_filtered_H1 = hampel(pupilRawRight_hot_woNan_dark_H1, winSize, sd)

# hand2
pupilLeft_hot_dark_filtered_H2 = hampel(pupilRawLeft_hot_woNan_dark_H2, winSize, sd)
pupilRight_hot_dark_filtered_H2 = hampel(pupilRawRight_hot_woNan_dark_H2, winSize, sd)

In [95]:
# plot the filtered signal for hot cup for hand1 in the dark and mark the logged live events
fig = plt.figure()

ax = fig.add_subplot(111)

ax.plot(time_hot_woNan_dark_plot_H1, pupilLeft_hot_dark_filtered_H1, label = 'left')
ax.plot(time_hot_woNan_dark_plot_H1, pupilRight_hot_dark_filtered_H1, label = 'right')

plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Pupil size [in mm]')

ax.set_title('Subject: %s -- Pupil size from hot cup for hand1 in the dark' %subjectName)

for ind in indices_loggedEvents_hot_dark_H1:
    ax.plot([timeMilliS[ind]/1000, timeMilliS[ind]/1000], [np.min(pupilLeft_hot_dark_filtered_H1), np.max(pupilLeft_hot_dark_filtered_H1)], '-.', color = 'gray')

pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName +  '\\pupils_hot_dark_wLogEventsMarked_H1', 'wb'))


In [96]:
# plot the filtered signal for hot cup for hand2 in the dark and mark the logged live events
fig = plt.figure()

ax = fig.add_subplot(111)

ax.plot(time_hot_woNan_dark_plot_H2, pupilLeft_hot_dark_filtered_H2, label = 'left')
ax.plot(time_hot_woNan_dark_plot_H2, pupilRight_hot_dark_filtered_H2, label = 'right')

plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Pupil size [in mm]')

ax.set_title('Subject: %s -- Pupil size from hot cup for hand2 in the dark' %subjectName)

for ind in indices_loggedEvents_hot_dark_H2:
    ax.plot([timeMilliS[ind]/1000, timeMilliS[ind]/1000], [np.min(pupilLeft_hot_dark_filtered_H2), np.max(pupilLeft_hot_dark_filtered_H2)], '-.', color = 'gray')

pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName +  '\\pupils_hot_dark_wLogEventsMarked_H2', 'wb'))


## Pupil sizes from pins in light

In [97]:

# pupil sizes for pain from pin 
pupilRawLeft_pin_light = pupilRawLeft[index_pin_start[-1]:index_pin_end[-1]]
pupilRawRight_pin_light = pupilRawRight[index_pin_start[-1]:index_pin_end[-1]]
time_pin_light = timeMilliS[index_pin_start[-1]:index_pin_end[-1]]

# pupil sizes from pin pain without nan
pupilRawLeft_pin_woNan_light = [pupilSize for ind, pupilSize in enumerate(pupilRawLeft_pin_light) if not np.isnan(pupilSize) and not np.isnan(pupilRawRight_pin_light[ind])]
pupilRawRight_pin_woNan_light = [pupilSize for ind, pupilSize in enumerate(pupilRawRight_pin_light) if not np.isnan(pupilSize) and not np.isnan(pupilRawLeft_pin_light[ind])]
time_pin_woNan_light = [time_pin_light[ind] for ind, pupilSize in enumerate(pupilRawLeft_pin_light) if not np.isnan(pupilSize) and not np.isnan(pupilRawRight_pin_light[ind])]

time_pin_woNan_light_plot = [time/1000 for time in time_pin_woNan_light]

In [98]:
# find the logged events in the pin events

for index in indices_loggedEvents_all:
    if index > index_pin_start[-1]:
        logged_events_start = index
        break
        
for index in sorted(indices_loggedEvents_all, reverse=True):
    if index < index_pin_end[-1]:
        logged_events_end = index
        break
        
indices_loggedEvents_pin_light = indices_loggedEvents_all[indices_loggedEvents_all.index(logged_events_start):indices_loggedEvents_all.index(logged_events_end)+1]

In [99]:
# filter the pupil size for pins in the light using hampel filter 

winSize = 35
sd = 3

pupilLeft_pin_light_filtered = hampel(pupilRawLeft_pin_woNan_light, winSize, sd)
pupilRight_pin_light_filtered = hampel(pupilRawRight_pin_woNan_light, winSize, sd)

In [100]:
# plot the filtered signal for pins in the light and mark the logged live events
fig = plt.figure()

ax = fig.add_subplot(111)

ax.plot(time_pin_woNan_light_plot, pupilLeft_pin_light_filtered, label = 'left')
ax.plot(time_pin_woNan_light_plot, pupilRight_pin_light_filtered, label = 'right')

plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Pupil size [in mm]')

ax.set_title('Subject: %s -- Pupil size from pins in the light' %subjectName)

for ind in indices_loggedEvents_pin_light:
    ax.plot([timeMilliS[ind]/1000, timeMilliS[ind]/1000], [np.min(pupilLeft_pin_light_filtered), np.max(pupilLeft_pin_light_filtered)], '-.', color = 'gray')

pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName +  '\\pupils_pin_light_wLogEventsMarked', 'wb'))


## Pupil sizes from hot cup in the light

In [101]:
# indexes for the pain using hot cup in the light

# indexes of hot events
index_hot_start =  [ind for ind, row in enumerate(events) if row == 'hot_start']
index_hot_end = [ind for ind, row in enumerate(events) if row == 'hot_end']


# hand1
# indexes of hot events for hand1
index_hot_start_H1 = [index for ind, index in enumerate(index_hot_start) if hot_indices[ind] == '1'][-1]
index_hot_end_H1 = [index for ind, index in enumerate(index_hot_end) if hot_indices[ind] == '1'][-1]

# pupil sizes for pain from hot for hand1
pupilRawLeft_hot_light_H1 = pupilRawLeft[index_hot_start_H1:index_hot_end_H1]
pupilRawRight_hot_light_H1 = pupilRawRight[index_hot_start_H1:index_hot_end_H1]
time_hot_light_H1 = timeMilliS[index_hot_start_H1:index_hot_end_H1]

# pupil sizes from hot pain without nan for hand1
pupilRawLeft_hot_woNan_light_H1 = [pupilSize for ind, pupilSize in enumerate(pupilRawLeft_hot_light_H1) if not np.isnan(pupilSize) and not np.isnan(pupilRawRight_hot_light_H1[ind])]
pupilRawRight_hot_woNan_light_H1 = [pupilSize for ind, pupilSize in enumerate(pupilRawRight_hot_light_H1) if not np.isnan(pupilSize) and not np.isnan(pupilRawLeft_hot_light_H1[ind])]
time_hot_woNan_light_H1 = [time_hot_light_H1[ind] for ind, pupilSize in enumerate(pupilRawLeft_hot_light_H1) if not np.isnan(pupilSize) and not np.isnan(pupilRawRight_hot_light_H1[ind])]

time_hot_woNan_light_plot_H1 = [time/1000 for time in time_hot_woNan_light_H1]


# hand2
# indexes of hot events for hand2
index_hot_start_H2 = [index for ind, index in enumerate(index_hot_start) if hot_indices[ind] == '2'][-1]
index_hot_end_H2 = [index for ind, index in enumerate(index_hot_end) if hot_indices[ind] == '2'][-1]

# pupil sizes for pain from hot for hand2
pupilRawLeft_hot_light_H2 = pupilRawLeft[index_hot_start_H2:index_hot_end_H2]
pupilRawRight_hot_light_H2 = pupilRawRight[index_hot_start_H2:index_hot_end_H2]
time_hot_light_H2 = timeMilliS[index_hot_start_H2:index_hot_end_H2]

# pupil sizes from hot pain without nan for hand2
pupilRawLeft_hot_woNan_light_H2 = [pupilSize for ind, pupilSize in enumerate(pupilRawLeft_hot_light_H2) if not np.isnan(pupilSize) and not np.isnan(pupilRawRight_hot_light_H2[ind])]
pupilRawRight_hot_woNan_light_H2 = [pupilSize for ind, pupilSize in enumerate(pupilRawRight_hot_light_H2) if not np.isnan(pupilSize) and not np.isnan(pupilRawLeft_hot_light_H2[ind])]
time_hot_woNan_light_H2 = [time_hot_light_H2[ind] for ind, pupilSize in enumerate(pupilRawLeft_hot_light_H2) if not np.isnan(pupilSize) and not np.isnan(pupilRawRight_hot_light_H2[ind])]

time_hot_woNan_light_plot_H2 = [time/1000 for time in time_hot_woNan_light_H2]




In [102]:
# find the logged events in the hot events in the light

# hand1
for index in indices_loggedEvents_all:
    if index > index_hot_start_H1:
        logged_events_start_H1 = index
        break
        
for index in sorted(indices_loggedEvents_all, reverse=True):
    if index < index_hot_end_H1:
        logged_events_end_H1 = index
        break

indices_loggedEvents_hot_light_H1 = indices_loggedEvents_all[indices_loggedEvents_all.index(logged_events_start_H1):indices_loggedEvents_all.index(logged_events_end_H1)+1]


# hand2
for index in indices_loggedEvents_all:
    if index > index_hot_start_H2:
        logged_events_start_H2 = index
        break
        
for index in sorted(indices_loggedEvents_all, reverse=True):
    if index < index_hot_end_H2:
        logged_events_end_H2 = index
        break
        
indices_loggedEvents_hot_light_H2 = indices_loggedEvents_all[indices_loggedEvents_all.index(logged_events_start_H2):indices_loggedEvents_all.index(logged_events_end_H2)+1]


In [103]:
# filter the pupil size for hot cup in the light using hampel filter 

winSize = 35
sd = 3

# hand1
pupilLeft_hot_light_filtered_H1 = hampel(pupilRawLeft_hot_woNan_light_H1, winSize, sd)
pupilRight_hot_light_filtered_H1 = hampel(pupilRawRight_hot_woNan_light_H1, winSize, sd)

# hand2
pupilLeft_hot_light_filtered_H2 = hampel(pupilRawLeft_hot_woNan_light_H2, winSize, sd)
pupilRight_hot_light_filtered_H2 = hampel(pupilRawRight_hot_woNan_light_H2, winSize, sd)

In [104]:
# plot the filtered signal for hot cup for hand1 in the light and mark the logged live events
fig = plt.figure()

ax = fig.add_subplot(111)

ax.plot(time_hot_woNan_light_plot_H1, pupilLeft_hot_light_filtered_H1, label = 'left')
ax.plot(time_hot_woNan_light_plot_H1, pupilRight_hot_light_filtered_H1, label = 'right')

plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Pupil size [in mm]')

ax.set_title('Subject: %s -- Pupil size from hot cup for hand1 in the light' %subjectName)

for ind in indices_loggedEvents_hot_light_H1:
    ax.plot([timeMilliS[ind]/1000, timeMilliS[ind]/1000], [np.min(pupilLeft_hot_light_filtered_H1), np.max(pupilLeft_hot_light_filtered_H1)], '-.', color = 'gray')

pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName +  '\\pupils_hot_light_wLogEventsMarked_H1', 'wb'))


In [105]:
# plot the filtered signal for hot cup for hand2 in the light and mark the logged live events
fig = plt.figure()

ax = fig.add_subplot(111)

ax.plot(time_hot_woNan_light_plot_H2, pupilLeft_hot_light_filtered_H2, label = 'left')
ax.plot(time_hot_woNan_light_plot_H2, pupilRight_hot_light_filtered_H2, label = 'right')

plt.legend()

ax.set_xlabel('Time [in s]')
ax.set_ylabel('Pupil size [in mm]')

ax.set_title('Subject: %s -- Pupil size from hot cup for hand2 in the light' %subjectName)

for ind in indices_loggedEvents_hot_light_H2:
    ax.plot([timeMilliS[ind]/1000, timeMilliS[ind]/1000], [np.min(pupilLeft_hot_light_filtered_H2), np.max(pupilLeft_hot_light_filtered_H2)], '-.', color = 'gray')

pickle.dump(fig, open(r'C:\DTU\Results\201907_PainTrial2' + '\\' + subjectName +  '\\pupils_hot_light_wLogEventsMarked_H2', 'wb'))
